# Idea


Semi supervised data preparation 

1. Build a classifier or use ResNet without image-net weights for training a classifier 
2. Freeze the classifier weights and build a segmentation model 


The data preparation for the classifier. Select 2 classes - Stroma , Tumor

For data preparation for the classifier - using the mask and image name see if the classes are seen on the image and using pixel altering keep the region coresponding to the mask as such for the image and the rest of the region pixelated. 

Create a dataset like for 500 each or 100 each for stroma and tumor in their directories. 

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

Te pixels with “Water” classes
are labeled as 1, “Building No Damage” as 2, “Building Minor Damage” as 3, “Building Major Damage” as 4,
“Building Total Destruction” as 5, “Vehicle” as 6, “Road-Clear” as 7, “Road-Blocked” as 8, “Tree” as 9, and “Pool”
as 10. Rest of the pixels are considered “Background” and labeled as 0.

In [2]:
def process_and_save_images(data_path, resize_shape, save_folder):
    images_path = os.path.join(data_path, "train-org-img")
    masks_path = os.path.join(data_path, "train-label-img")
    
    file_list = os.listdir(images_path)

    # Create directories for output once
    damage_types = ['background', 'water', 'building_no_damage', 'building_minor_damage','building_major_damage','building_destroyed','vehicle','road_clear','road_blocked','tree','pool']
    for damage_type in damage_types:
        os.makedirs(os.path.join(save_folder, damage_type), exist_ok=True)

    for file_name in tqdm(file_list):
        image_path = os.path.join(images_path, file_name)
        mask_path = os.path.join(masks_path, file_name.replace('.jpg', '_lab.png'))
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, resize_shape, interpolation=cv2.INTER_LINEAR)

        mask = cv2.imread(mask_path, 0)  # Read mask as grayscale
        mask = cv2.resize(mask, resize_shape, interpolation=cv2.INTER_LINEAR)

        # Save class labels
        for i, label in enumerate(damage_types):
            if (mask == i).any():
                blank_image = np.zeros_like(image)
                coords = np.argwhere(mask == i)
                for x, y in coords:
                    blank_image[x, y] = image[x, y]
                Image.fromarray(blank_image).save(os.path.join(save_folder, label, file_name))

def apply_preprocessing(data_path, resize_shape, save_folder):
    process_and_save_images(data_path, resize_shape, save_folder)
    print('Saved Classifier Processed Dataset')


In [3]:
# Build Train Dataset 
DATA_PATH = "/home/aaimscadmin/IRP_DATA/RescueNet/train"
SAVE_FOLDER = "/home/aaimscadmin/IRP_DATA/RescueNet/classifier_dataset/train"

RESIZE_SHAPE=(256,256)
apply_preprocessing(DATA_PATH,RESIZE_SHAPE,SAVE_FOLDER)

100%|██████████| 3595/3595 [1:18:53<00:00,  1.32s/it]

Saved Classifier Processed Dataset


In [4]:
def process_and_save_images(data_path, resize_shape, save_folder):
    images_path = os.path.join(data_path, "val-org-img")
    masks_path = os.path.join(data_path, "val-label-img")
    
    file_list = os.listdir(images_path)

    # Create directories for output once
    damage_types = ['background', 'water', 'building_no_damage', 'building_minor_damage','building_major_damage','building_destroyed','vehicle','road_clear','road_blocked','tree','pool']
    for damage_type in damage_types:
        os.makedirs(os.path.join(save_folder, damage_type), exist_ok=True)

    for file_name in tqdm(file_list):
        image_path = os.path.join(images_path, file_name)
        mask_path = os.path.join(masks_path, file_name.replace('.jpg', '_lab.png'))
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, resize_shape, interpolation=cv2.INTER_LINEAR)

        mask = cv2.imread(mask_path, 0)  # Read mask as grayscale
        mask = cv2.resize(mask, resize_shape, interpolation=cv2.INTER_LINEAR)

        # Save class labels
        for i, label in enumerate(damage_types):
            if (mask == i).any():
                blank_image = np.zeros_like(image)
                coords = np.argwhere(mask == i)
                for x, y in coords:
                    blank_image[x, y] = image[x, y]
                Image.fromarray(blank_image).save(os.path.join(save_folder, label, file_name))

def apply_preprocessing(data_path, resize_shape, save_folder):
    process_and_save_images(data_path, resize_shape, save_folder)
    print('Saved Classifier Processed Dataset')


In [5]:
# Build Train Dataset 
DATA_PATH = "/home/aaimscadmin/IRP_DATA/RescueNet/val"
SAVE_FOLDER = "/home/aaimscadmin/IRP_DATA/RescueNet/classifier_dataset/val"

RESIZE_SHAPE=(256,256)
apply_preprocessing(DATA_PATH,RESIZE_SHAPE,SAVE_FOLDER)

100%|██████████| 449/449 [09:39<00:00,  1.29s/it]

Saved Classifier Processed Dataset
